# Getting started with Integrated Hybrid Computing
In this sample, you will discover how to blend classical and quantum instructions in the same program, all fully processed by the quantum computing backend.
You can learn more about integrated hybrid computing at https://aka.ms/AQ/Hybrid/Docs

## Connect to the Azure Quantum workspace
First, find the resource ID of your Azure Quantum workspace. You can copy/paste this from the top-right corner of your Quantum Workspace page in the Azure Portal.

Next, you can run `%azure.connect` to connect to the workspace and display the list of provisioned targets that support execution of Q# programs. If you are prompted to login, be sure to use the same account you used to create your Azure Quantum workspace.

In [ ]:
%azure.connect "/subscriptions/subscriptionId/resourceGroups/resourceGroupName/providers/Microsoft.Quantum/Workspaces/workspaceName" location="westus"

## Select a target and opt-in to use integrated hybrid computing
First, we select the target the program is intended to be run on using the `%azure.target` command. This information is used by Q# compiler to check the program can be run on the specified target taking into consideration its capabilities.

Since integrated hybrid computing is an opt-in feature, we run the `%azure.target-capability` command to opt into it.


In [ ]:
%azure.target quantinuum.sim.h1-1e
%azure.target-capability AdaptiveExecution

## Define the CheckGHZ operation

This program prepares a Greenberger–Horne–Zeilinger (GHZ) state and counts the number of times qubit measurements are not perfectly correlated out of 10 attempts. Unlike in noiseless simulations, non-perfect correlations can occur when programs run on real quantum devices which are subject to the effects of noise.

In the CheckGHZ() operation, we can see two capabilities of integrated hybrid computing being demonstrated:
1. **Branching** based on qubit measurements, which allows programs to execute different quantum operations or classical instructions depending on the results yielded by qubit measurements.
```Q#
   if not (r0 == r1 and r1 == r2)
```
2. **Classical compute** performed in real time by the quantum machine itself.
```Q#
   set mismatch += 1;
```
### ❕ Please note that you will see a warning displayed (QS5028)
It can be ignored for the purposes of this sample. It will be addressed in a subsequent release of the Quantum Development Kit.

In [ ]:
open Microsoft.Quantum.Intrinsic;
open Microsoft.Quantum.Measurement;

operation CheckGHZ() : Int {
    use q = Qubit[3];
    mutable mismatch = 0;
    for _ in 1..10 {
        // Prepare the GHZ state.
        H(q[0]);
        CNOT(q[0], q[1]);
        CNOT(q[1], q[2]);

        // Measures and resets the 3 qubits
        let (r0, r1, r2) = (MResetZ(q[0]), MResetZ(q[1]), MResetZ(q[2]));

        // Adjusts classical value based on qubit measurement results
        if not (r0 == r1 and r1 == r2) {
            set mismatch += 1;
        }
    }
    return mismatch;
}

## Submit the program
### ❕ This program will use approximately 10.65 units (HQCs or eHQCs) using 50 shots, as configured below.
If the job has not been completed after the client times out (300 seconds), you can query the status using `%azure.status` followed by querying the output using `%azure.output`.<br/>

If the kernel was restarted after the program had been submitted, you will have to use `%azure.status <job_id>` to get the status of the job and `%azure.output <job_id>` to retrieve the results.<br/>
You can also check the status of the job under the **Job management** section on the left side menu of the portal.


In [ ]:
%azure.execute CheckGHZ shots=50 timeout=300 poll=10

In [ ]:
%azure.status

## Understanding the results
Each item in the histogram represents the correlation mismatch count and the corresponding frequency the mismatch count was observed out of the number of shots that were run.<br/>
The histogram values can range from 0 (no correlation mismatch occurred) to 10 (all correlations checks were a mismatch).

In [ ]:
%azure.output

## ↗ Learning more about Integrated Hybrid Computing
Now that you have run this sample, you can further learn and experiment with other samples available in the gallery.

To learn more, please refer to our [online documentation](https://aka.ms/AQ/Hybrid/Docs).